<a href="https://colab.research.google.com/github/Prakzter/NDSCcompetition2019/blob/master/1stdraft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Unzipping image files

In [0]:
from google.colab import drive
import os
import tarfile

drive.mount('/content/gdrive')

path = os.path.join(os.getcwd(), 'gdrive', 'My Drive', 'NDSC')   # Change according to your path 

os.chdir(path)

with tarfile.open('beauty_image.tar.gz', 'r:gz') as tar:
    tar.extractall()


Importing libraries

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

Next, we will read all the training images, store them in a list, and finally convert that list into a numpy array.

In [0]:
train = pd.read_csv('train.csv')

# We have grayscale images, so while loading the images we will keep grayscale=True, if you have RGB images, you should set grayscale as False
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img('train/'+train['id'][i].astype('str')+'.png', target_size=(28,28,3), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

As it is a multi-class classification problem (57 classes), we will one-hot encode the target variable.

In [0]:
y = train['label'].values
y = to_categorical(y)

Creating a validation set from the training data.

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

Define the model structure.

We will create a simple architecture with 2 convolutional layers, one dense hidden layer and an output layer.

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

Next, we will compile the model we’ve created.

In [0]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])


Training the model.

In this step, we will train the model on the training set images and validate it using, you guessed it, the validation set.

In [0]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Making predictions!

We’ll initially follow the steps we performed when dealing with the training data. Load the test images and predict their classes using the model.predict_classes() function.

In [0]:
test = pd.read_csv('test.csv')

test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img('test/'+test['id'][i].astype('str')+'.png', target_size=(28,28,1), grayscale=True)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)

Let's make our prediction (and cross our fingers!)

In [0]:
prediction = model.predict_classes(test)

Creating final submission file

In [0]:
sample = pd.read_csv('Sample_Submission_lxuyBuB.csv')
sample['filename'] = test_file['filename']
sample['label'] = prediction
sample.to_csv('sample.csv', header=True, index=False)